In [1]:
import pandas as pd
import numpy as np

If you want to get a single value for each group, use aggregate() (or one of its shortcuts). <br>
If you want to get a subset of the original rows, use filter(). <br>
And if you want to get a new value for each original row, use transform().

Data contains different daily objects, with shapes you can superimpose on them, their colour, and how many of each item is present

In [2]:
objects_data = pd.read_csv('datasets/objects.csv')

In [3]:
objects_data

,object,shape,colour,quantity
0,mirror,circle,silver,1
1,wheel,circle,silver,4
2,lemon,circle,yellow,4
3,box,square,brown,3
4,bread,square,white,10
5,towel,rectangle,white,2
6,laptop,rectangle,black,1
7,pen,rectangle,blue,7
8,bottle,rectangle,red,6
9,biscuit,rectangle,brown,20


Grouping the data by shape of the object

In [4]:
shape_data = objects_data.groupby('shape')

shape_data

Looking at the first set of objects for each shape

In [5]:
shape_data.first()

,object,colour,quantity
shape,,,
circle,mirror,silver,1
rectangle,towel,white,2
square,box,brown,3


looking at all square objects

In [6]:
shape_data.get_group('square')

,colour,object,quantity
3,brown,box,3
4,white,bread,10


summing over the quantities of all objects, according to shape

In [7]:
shape_data.quantity.sum()

shape
circle        9
rectangle    36
square       13
Name: quantity, dtype: int64

classifying objects by colour and finding the average number of objects in each colour

In [8]:
colour_data = objects_data.groupby('colour')

In [9]:
colour_data.quantity.mean()

colour
black      1.0
blue       7.0
brown     11.5
red        6.0
silver     2.5
white      6.0
yellow     4.0
Name: quantity, dtype: float64

Note that the colours listed above are in alphabetical order. Groupby automatically parses data in this manner. 

To speed up processing, or in order to observe data in first come first serve basis, 

In [10]:
colour_data = objects_data.groupby(['colour'],
                                   sort=False)
colour_data.quantity.mean()

colour
silver     2.5
yellow     4.0
brown     11.5
white      6.0
black      1.0
blue       7.0
red        6.0
Name: quantity, dtype: float64

#### Convert this dataset to a multiindex one to explore further groupby operations
First transpose the dataframe and then convert to a 2D list 

In [11]:
array = objects_data.values.T.tolist()

In [12]:
array

[['mirror',
  'wheel',
  'lemon',
  'box',
  'bread',
  'towel',
  'laptop',
  'pen',
  'bottle',
  'biscuit'],
 ['circle',
  'circle',
  'circle',
  'square',
  'square',
  'rectangle',
  'rectangle',
  'rectangle',
  'rectangle',
  'rectangle'],
 ['silver',
  'silver',
  'yellow',
  'brown',
  'white',
  'white',
  'black',
  'blue',
  'red',
  'brown'],
 [1, 4, 4, 3, 10, 2, 1, 7, 6, 20]]

In [13]:
array_index = array[1:3]

In [14]:
array_index

[['circle',
  'circle',
  'circle',
  'square',
  'square',
  'rectangle',
  'rectangle',
  'rectangle',
  'rectangle',
  'rectangle'],
 ['silver',
  'silver',
  'yellow',
  'brown',
  'white',
  'white',
  'black',
  'blue',
  'red',
  'brown']]

In [15]:
index = pd.MultiIndex.from_arrays(array_index,
                                  names = ['shape', 'colour'])

In [16]:
multi_data = pd.DataFrame({'object':array[0],
                           'quantity':array[3]}, 
                            index = index)

In [17]:
multi_data

object  quantity
shape     colour                   
circle    silver   mirror         1
          silver    wheel         4
          yellow    lemon         4
square    brown       box         3
          white     bread        10
rectangle white     towel         2
          black    laptop         1
          blue        pen         7
          red      bottle         6
          brown   biscuit        20

In [18]:
multi_data.groupby(level=['shape']).sum()

,quantity
shape,
circle,9
rectangle,36
square,13


In [19]:
multi_data.groupby(level=['colour']).sum()

,quantity
colour,
black,1
blue,7
brown,23
red,6
silver,5
white,12
yellow,4


### grouping by multiple levels

In [20]:
multi_data.groupby(level=['shape','colour']).sum()

quantity
shape     colour          
circle    silver         5
          yellow         4
rectangle black          1
          blue           7
          brown         20
          red            6
          white          2
square    brown          3
          white         10

Trying the reverse multi level grouping

In [21]:
multi_data.groupby(level=['colour','shape']).sum()

quantity
colour shape              
black  rectangle         1
blue   rectangle         7
brown  rectangle        20
       square            3
red    rectangle         6
silver circle            5
white  rectangle         2
       square           10
yellow circle            4

Getting a specific element from our groupby operation

In [22]:
multi_data.groupby(level=['colour','shape'])\
          .get_group(('white','rectangle'))

,,object,quantity
shape,colour,,
rectangle,white,towel,2


### Grouping by one index and one column

In [23]:
multi_data.groupby([pd.Grouper(level='shape'), 'object']).sum()

quantity
shape     object           
circle    lemon           4
          mirror          1
          wheel           4
rectangle biscuit        20
          bottle          6
          laptop          1
          pen             7
          towel           2
square    box             3
          bread          10

The same can also be specified using level number as defined in the multiindex dataframe, instead of specifying the level name explicitly

In [24]:
multi_data.groupby([pd.Grouper(level=0), 'object']).sum()

quantity
shape     object           
circle    lemon           4
          mirror          1
          wheel           4
rectangle biscuit        20
          bottle          6
          laptop          1
          pen             7
          towel           2
square    box             3
          bread          10

### Iterating through these groups

In [25]:
for name,group in objects_data.groupby('colour'):
    print ('Name:', name)
    print('Group:', group)
    print()

Name: black
Group:    object      shape colour  quantity
6  laptop  rectangle  black         1

Name: blue
Group:   object      shape colour  quantity
7    pen  rectangle   blue         7

Name: brown
Group:     object      shape colour  quantity
3      box     square  brown         3
9  biscuit  rectangle  brown        20

Name: red
Group:    object      shape colour  quantity
8  bottle  rectangle    red         6

Name: silver
Group:    object   shape  colour  quantity
0  mirror  circle  silver         1
1   wheel  circle  silver         4

Name: white
Group:   object      shape colour  quantity
4  bread     square  white        10
5  towel  rectangle  white         2

Name: yellow
Group:   object   shape  colour  quantity
2  lemon  circle  yellow         4



Checking the size of the groups

In [26]:
multi_data.groupby(level=['colour','shape']).size()

colour  shape    
black   rectangle    1
blue    rectangle    1
brown   rectangle    1
        square       1
red     rectangle    1
silver  circle       2
white   rectangle    1
        square       1
yellow  circle       1
dtype: int64

You can also get a description of the grouped data

In [27]:
multi_data.groupby(level=['colour','shape']).describe()

quantity                                               
                    count  mean      std   min    25%   50%    75%   max
colour shape                                                            
black  rectangle      1.0   1.0      NaN   1.0   1.00   1.0   1.00   1.0
blue   rectangle      1.0   7.0      NaN   7.0   7.00   7.0   7.00   7.0
brown  rectangle      1.0  20.0      NaN  20.0  20.00  20.0  20.00  20.0
       square         1.0   3.0      NaN   3.0   3.00   3.0   3.00   3.0
red    rectangle      1.0   6.0      NaN   6.0   6.00   6.0   6.00   6.0
silver circle         2.0   2.5  2.12132   1.0   1.75   2.5   3.25   4.0
white  rectangle      1.0   2.0      NaN   2.0   2.00   2.0   2.00   2.0
       square         1.0  10.0      NaN  10.0  10.00  10.0  10.00  10.0
yellow circle         1.0   4.0      NaN   4.0   4.00   4.0   4.00   4.0

#### Aggregation

In [28]:
multi_data.groupby(level=['shape']).aggregate(np.sum)

,quantity
shape,
circle,9
rectangle,36
square,13


#### Two-level aggregation

In [29]:
multi_data.groupby(level=['shape','colour']).aggregate(np.sum)

quantity
shape     colour          
circle    silver         5
          yellow         4
rectangle black          1
          blue           7
          brown         20
          red            6
          white          2
square    brown          3
          white         10

If you do not want a multiindex dataframe as the result,

In [30]:
multi_data.groupby(level=['shape'], 
                   as_index=False).agg(np.sum)

,quantity
0,9
1,36
2,13


In [31]:
multi_data.groupby(level=['shape', 'colour'], 
                   as_index=False).agg(np.sum)

,quantity
0,5
1,4
2,1
3,7
4,20
5,6
6,2
7,3
8,10


Performing multiple aggregations at once

In [32]:
multi_data.groupby(level=['shape'])\
          .agg([np.sum, np.mean, np.min, np.max])

quantity               
               sum mean amin amax
shape                            
circle           9  3.0    1    4
rectangle       36  7.2    1   20
square          13  6.5    3   10

Performing aggregations on different columns

In [33]:
multi_data.groupby(level=['shape'])\
          .agg({'quantity' : np.sum, 
                'object' : np.max})

,quantity,object
shape,,
circle,9,wheel
rectangle,36,towel
square,13,bread


### Transformations

In [34]:
multi_data.groupby(level=['shape']).transform(lambda x:x.mean())

quantity
shape     colour          
circle    silver       3.0
          silver       3.0
          yellow       3.0
square    brown        6.5
          white        6.5
rectangle white        7.2
          black        7.2
          blue         7.2
          red          7.2
          brown        7.2

Assigning this to a new column in the dataframe

In [35]:
multi_data['average'] = multi_data\
                        .groupby(level=['shape'])\
                        .transform(lambda x:x.mean())

In [36]:
multi_data

object  quantity  average
shape     colour                            
circle    silver   mirror         1      3.0
          silver    wheel         4      3.0
          yellow    lemon         4      3.0
square    brown       box         3      6.5
          white     bread        10      6.5
rectangle white     towel         2      7.2
          black    laptop         1      7.2
          blue        pen         7      7.2
          red      bottle         6      7.2
          brown   biscuit        20      7.2

Notice how the average is added for every row in the dataframe. Groupby gives only one entry for every group. net every row. 